## Setup and Imports

In [1]:
import os
os.chdir("C:/Users/yotam/code_projects/APDTFlow")
print("New working directory:", os.getcwd())

New working directory: C:\Users\yotam\code_projects\APDTFlow


In [2]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)


Project root added to sys.path: C:\Users\yotam


In [3]:
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

torch.manual_seed(42)

from apdtflow.models.apdtflow import APDTFlow
from apdtflow.data import TimeSeriesWindowDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## Load and Prepare the Dataset

In [4]:
csv_file = r"C:\Users\yotam\code_projects\APDTFlow\dataset_examples\Electric_Production.csv"

dataset = TimeSeriesWindowDataset(
    csv_file=csv_file,
    date_col="DATE",
    value_col="IPG2211A2N",
    T_in=12, 
    T_out=3  
)
print("Loaded dataset with", len(dataset), "samples.")

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

Loaded dataset with 383 samples.


## Instantiate the APDTFlow Model with Embedding Enabled

In [5]:
num_scales = 3
input_channels = 1
filter_size = 5
hidden_dim = 16
output_dim = 1
forecast_horizon = 3

model = APDTFlow(
    num_scales=num_scales,
    input_channels=input_channels,
    filter_size=filter_size,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    forecast_horizon=forecast_horizon,
    use_embedding=True 
)

model.to(device)
print("Model instantiated and moved to device:", device)

Model instantiated and moved to device: cpu


## Train the Model

In [6]:
num_epochs = 5     
learning_rate = 0.001

print("Starting training...")
model.train_model(train_loader, num_epochs=num_epochs, learning_rate=learning_rate, device=device)

print("Training completed.")

Starting training...
Epoch 1/5, Loss: 1820.2332
Epoch 2/5, Loss: 615.7756
Epoch 3/5, Loss: 408.1793
Epoch 4/5, Loss: 305.1142
Epoch 5/5, Loss: 238.4244
Training completed.


## Evaluate the Model

In [7]:
metrics = model.evaluate(train_loader, device, metrics=["MSE", "MAE"])
print("Evaluation metrics on training data:", metrics)

2025-02-09 13:32:12,671 - INFO - Evaluation -> MSE: 8137.4237, MAE: 88.9985


Evaluation metrics on training data: {'MSE': 8137.42368023825, 'MAE': 88.99845914118595}
